In [1]:
'''
Installing packages for rendering the game on Colab
'''

!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install git+https://github.com/tensorflow/docs > /dev/null 2>&1
!pip install gym[classic_control]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.2
    Uninstalling pygame-2.5.2:
      Successfully uninstalled pygame-2.5.2


In [ ]:
!pip install wandb
import wandb
# Replace with your actual API key
api_key = "17dab9d1bbdc37c41831799a4b0b50d3e97400c5"

# Login to Weights & Biases
wandb.login(key=api_key)
project_name = 'D1CS2'

In [ ]:
!pip install torch

In [4]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import namedtuple, deque
import torch.optim as optim
import datetime
import gym
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
import matplotlib.pyplot as plt
from IPython.display import HTML
from pyvirtualdisplay import Display
import tensorflow as tf
from IPython import display as ipythondisplay
from PIL import Image
import tensorflow_probability as tfp
import wandb

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [5]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:

# Define the Dueling DQN model
class DuelingDQN(nn.Module):
    def __init__(self, input_size, output_size, fc1_units, fc2_units, seed):
        super(DuelingDQN, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(input_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.advantage = nn.Linear(fc2_units, output_size)
        self.value = nn.Linear(fc2_units, 1)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        advantage = self.advantage(x)
        value = self.value(x)
        q_values = value + (advantage - advantage.mean(dim=-1, keepdim=True))
        return q_values


In [7]:
# Define the ReplayBuffer class
class ReplayBuffer:
    def __init__(self, action_size, buffer_size, batch_size, seed):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        return len(self.memory)


In [8]:
# Define the agent
class Agent():
    def __init__(self, state_size, action_size, fc1_units, fc2_units, buffer_size, batch_size, lr, update_every, gamma, eps_end, eps_decay, seed):
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = batch_size
        self.lr = lr
        self.update_every = update_every
        self.gamma = gamma
        self.seed = random.seed(seed)
        self.q_network = DuelingDQN(state_size, action_size, fc1_units, fc2_units, seed).to(device)
        self.target_network = DuelingDQN(state_size, action_size, fc1_units, fc2_units, seed).to(device)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        ''' Replay memory '''
        self.memory = ReplayBuffer(action_size, buffer_size, batch_size, seed)
        self.t_step = 0
        self.eps_end = eps_end
        self.eps_decay = eps_decay

    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)

        if len(self.memory) >= self.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences)

        self.t_step = (self.t_step + 1) % self.update_every
        if self.t_step == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())

    def act(self, state, eps=0.0):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.q_network.eval()
        with torch.no_grad():
            action_values = self.q_network(state)
        self.q_network.train()
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences):
        states, actions, rewards, next_states, dones = experiences
        Q_targets_next = self.target_network(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))
        Q_expected = self.q_network(states).gather(1, actions)
        loss = F.mse_loss(Q_expected, Q_targets)
        # wandb.log({'train loss': loss})
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.q_network.parameters():
            param.grad.data.clamp_(-1, 1)  # Gradient clipping
        self.optimizer.step()


In [9]:
def dqn(agent, env, n_episodes=10000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    scores_window = deque(maxlen=100)
    episode_list_epsgrdy = []
    average_scores_epsgrdy = []
    average_regret_epsgrdy = []
    cumulative_regret_epsgrdy = []
    cummulative_regret_window=[]
    regret_window = []
    cumulative_regret = 0  # Initialize cumulative regret
    eps = eps_start

    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        regret = 0  # Initialize regret for this episode
        # cumulative_regret = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            optimal_action = np.argmax(agent.q_network(torch.from_numpy(state).float().unsqueeze(0)).cpu().data.numpy())
            optimal_reward = env.step(optimal_action)[1]  # Get the reward for the optimal action
            regret += optimal_reward - reward  # Calculate regret for this time step
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward

            if done:
                break


        scores_window.append(score)
        average_score = np.mean(scores_window)
        average_scores_epsgrdy.append(average_score)


        regret_window.append(regret)
        average_regret = np.mean(regret_window)
        average_regret_epsgrdy.append(average_regret)

        cumulative_regret += average_regret  # Update cumulative regret

        # cummulative_regret_window.append(cumulative_regret)
        # cummulative_average_regret = np.mean(cummulative_regret_window)
        cummulative_average_regret = average_regret
        cumulative_regret_epsgrdy.append(cumulative_regret)

        episode_list_epsgrdy.append(i_episode)
        wandb.log({'average_score': average_score})
        wandb.log({'average_regret': average_regret})
        wandb.log({"cummulative_regret": cumulative_regret})

        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, average_score), end="")

        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}\tRegret: {:.2f}'.format(i_episode, average_score, regret))

        if np.mean(scores_window)>=195.0:
           print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
           break
        eps = max(eps_end, eps_decay*eps)

    return episode_list_epsgrdy, average_scores_epsgrdy, average_regret_epsgrdy

In [10]:

# Initialize wandb with your project name
wandb.init(project=project_name)

wandb: Currently logged in as: rishi1906 (iitm_aero). Use `wandb login --relogin` to force relogin


In [11]:
env = gym.make('CartPole-v1')
env.seed(0)
state_shape = env.observation_space.shape[0]
action_shape = env.action_space.n

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [12]:
# Sweep configuration
sweep_config = {
    "method": "random",
    "metric": {"goal": "minimize", "name": "cumulative_regret"},
    "parameters": {
        'state_size': {
            'values': [state_shape]
        },
        'action_size': {
            'values': [action_shape]
        },
        'BUFFER_SIZE': {
            'values': [int(1e5)]
            # 'values': [int(1e4), int(1e5), int(1e6)]
        },
        'BATCH_SIZE': {
            'values': [64]
            # 'values': [32, 64, 128, 256]
        },
        'LR': {
            'values': [0.0001]
            # 'values': [0.1, 0.01, 0.001, 0.0001]
        },
        'UPDATE_EVERY': {
            'values': [20]
            # 'values': [4, 6, 10, 20]
        },
        'fc1_units': {
            # 'values': [128]
            'values': [128]
            # 'values': [64, 128, 256]
        },
        'fc2_units': {
            # 'values': [64]
            'values': [64]
            # 'values': [64, 128, 256]
        },
        'eps_start': {
            'values': [1]
        },
        'eps_end': {
            'values': [0.01]
            # 'values': [0.01, 0.05, 0.1]
        },
        'eps_decay': {
            'values': [0.995]
            # 'values': [0.9, 0.95, 0.99, 0.995, 0.999]
        },
        'gamma': {
            'values': [0.99]
        },
        'n_episodes': {
            'values': [5000]
            # 'values': [1000, 2000, 5000]
        },
        'max_t': {
            'values': [1000]
            # 'values': [500, 1000, 2000]
        }
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project=project_name)

Create sweep with ID: i7h94k9t
Sweep URL: https://wandb.ai/iitm_aero/D1CS2/sweeps/i7h94k9t


In [13]:
average_scores = []
cumm_regret = []

In [14]:


# Define the main function for hyperparameter tuning
def AvgOverExperiments():

    # env = gym.make('Acrobot-v1')
    # env.seed(0)
    # state_shape = env.observation_space.shape[0]
    # no_of_actions = env.action_space.n

    with wandb.init() as run:
        # Get the hyperparameters for this run
        config = wandb.config

        begin_time = datetime.datetime.now()
        # env = gym.make('Acrobot-v1')
        # env.seed(0)

        # Create the agent with the hyperparameters
        agent = Agent(state_size=config.state_size, action_size=config.action_size, fc1_units=config.fc1_units, fc2_units=config.fc2_units, buffer_size=config.BUFFER_SIZE, batch_size=config.BATCH_SIZE, lr=config.LR, update_every=config.UPDATE_EVERY, gamma=config.gamma, eps_end=config.eps_end, eps_decay=config.eps_decay,seed = 0)

        # Train the agent and get the scores
        # episode_list_epsgrdy, average_scores_epsgrdy = dqn(agent, env, n_episodes=config.n_episodes, max_t=config.max_t, eps_start=config.eps_start, eps_end=config.eps_end, eps_decay=config.eps_decay)

        episode_list_epsgrdy, average_scores_epsgrdy, cumulative_regret_epsgrdy = dqn(agent, env, n_episodes=config.n_episodes, max_t=config.max_t, eps_start=config.eps_start, eps_end=config.eps_end, eps_decay=config.eps_decay)

        time_taken = datetime.datetime.now() - begin_time

        print(time_taken)
        # Log the final average score to wandb
        # wandb.log({"Average Score": average_scores_epsgrdy})
        # wandb.log({"Average Score": cumulative_regret})
        print(average_scores_epsgrdy)
        print('\n')
        print(cumulative_regret_epsgrdy)
        average_scores.append(average_scores_epsgrdy)
        cumm_regret.append(cumulative_regret_epsgrdy)
        # return [average_scores_epsgrdy, cumulative_regret]


In [15]:
def plot_avg_curves(avg_reward,reward_std, avg_step, step_std, step_size = 1 ):

    trunc_avg_reward = avg_reward[0::step_size]
    tunc_reward_std = reward_std[0::step_size]

    trunc_avg_step = avg_step[0::step_size]
    tunc_step_std = step_std[0::step_size]

    # Plot the average rewards with mean and standard deviation
    plt.figure()
    plt.plot(trunc_avg_reward, label='Mean')
    plt.fill_between(range(step_size), trunc_avg_reward + tunc_reward_std, trunc_avg_reward - tunc_reward_std, alpha=0.5, label=' Std Dev')
    plt.xlabel('Episode')
    plt.ylabel('Average Total Reward')
    plt.title('Average Total Reward vs Episode')
    plt.legend()
    # plt.savefig( "RvsE " +".pdf")
    # files.download(exp_no+"_RvsE.png")
    plt.show()



    # Plot the average rewards with mean and standard deviation
    plt.figure()
    plt.plot(trunc_avg_step, label='Mean')
    plt.fill_between(range(step_size), trunc_avg_step + tunc_step_std, trunc_avg_step - tunc_step_std, alpha=0.5, label=' Std Dev')
    plt.xlabel('Episode')
    plt.ylabel('Average Regret')
    plt.title(' Average Regret vs Episode')
    plt.legend()
    # plt.savefig("SvsE " + ".pdf")
    # files.download(exp_no+"_SvsE.png")
    plt.show()

    # Plot the average rewards with mean and standard deviation
    plt.figure()
    plt.plot(avg_step, label='Mean')
    # plt.fill_between(range(step_size), trunc_avg_step + tunc_step_std, trunc_avg_step - tunc_step_std, alpha=0.5, label=' Std Dev')
    plt.xlabel('Episode')
    plt.ylabel('Cummulative Regret')
    plt.title('Cummulative Average Regret vs Episode')
    plt.legend()
    # plt.savefig("SvsE " + ".pdf")
    # files.download(exp_no+"_SvsE.png")
    plt.show()


In [ ]:
# # Average over 5 exeriments

# average_scores, cumm_regret = [], []
num_expts = 5
for i in range(num_expts):
    print("Experiment: %d" % (i + 1))
    # episode_average_scores, episode_cumm_regret =  wandb.agent(sweep_id, function=AvgOverExperiments, count=1)
    wandb.agent(sweep_id, function=AvgOverExperiments, count=1)
    # average_scores.append(episode_average_scores)
    # cumm_regret.append(episode_cumm_regret)
wandb.finish()


In [ ]:
average_scores = np.array(average_scores)
cumm_regret = np.array(cumm_regret)

mean_average_scores = np.mean(average_scores, axis=0)
std_mean_average_scores = np.std(average_scores, axis=0)

mean_cumm_regret = np.mean(cumm_regret, axis=0)
mean_cumm_regret = np.std(cumm_regret, axis=0)

plot_avg_curves(mean_average_scores, std_mean_average_scores, mean_cumm_regret, mean_cumm_regret, step_size = 10)